En esta lección, crearemos un sistema RAG utilizando Cortex Search y Cortex LLM. A continuación, añadiremos el rastreo de OpenTelemetry con TruLens a la aplicación. Por último, crearemos un conjunto de pruebas y ejecutaremos evaluaciones de LLM como juez en batch en nuestra aplicación.

## ¿Qué es TruLens?

TruLens es una librería de código abierto para el seguimiento y la evaluación de aplicaciones de IA generativa, que además impulsa la Observabilidad de IA de Snowflake.

## Privilegios

Si usted no están trabajando con una cuenta de prueba de Snowflake ni tiene el rol de `ACCOUNTADMIN`, asegúrese de que el rol de usuario que esté usando tenga asignado los siguientes roles :

- Rol de base de datos `SNOWFLAKE.CORTEX_USER`
- Rol de aplicación `SNOWFLAKE.AI_OBSERVABILITY_EVENTS_LOOKUP`
- Privilegio `CREATE EXTERNAL AGENT` en el esquema

## Librerías requeridas

En su notebook de Snowflake, agregue las siguientes librerías de Python: 

- `snowflake-ml-python`
- `snowflake.core`
- `trulens-core`
- `trulens-providers-cortex`
- `trulens-connectors-snowflake`

A continuación, vamos a crear un esquema para esta sección del curso.

In [ ]:
CREATE OR REPLACE SCHEMA curso_ia.seccion_6;

In [ ]:
# Obtener la sesión de snowpark
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## RAG

Para esta lección vamos a utilizar el servicio de Cortex Search `aws_search_service` que construimos en la sección 4 del curso.

A continuación, podemos crear una clase `CortexSearchRetreiver` para conectarnos a nuestro servicio de búsqueda Cortex y agregar el método `retrieve` .

In [ ]:
import os
from snowflake.core import Root
from typing import List
from snowflake.snowpark.session import Session

class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(session)

        search_service = (root
          .databases["curso_ia"]
          .schemas["seccion_4"]
          .cortex_search_services["aws_search_service"]
        )
        resp = search_service.search(
          query=query,
          columns=["chunk"],
          limit=self._limit_to_retrieve
        )

        if resp.results:
            return [curr["chunk"] for curr in resp.results]
        else:
            return []

Probamos la clase que acabamos de crear realizando una consulta al servicio de Cortex.

In [ ]:
retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=2)

retrieved_context = retriever.retrieve(query="¿Cuáles servicios serverless de AWS tienes listado?")

retrieved_context

### Habilitar TruLens-OpenTelemetry para rastreo y observabilidad

In [ ]:
import os
os.environ["TRULENS_OTEL_TRACING"] = "1"

Luego, construyamos el sistema RAG con instrumentación integrada utilizando el `retrive` que creamos previamente.

In [ ]:
from snowflake.cortex import complete
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes

class RAG:
    def __init__(self):
        self.retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=4)
    
    @instrument(
        span_type=SpanAttributes.SpanType.RETRIEVAL,
        attributes={
            SpanAttributes.RETRIEVAL.QUERY_TEXT: "query",
            SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: "return",
            }
    )
    def retrieve_context(self, query: str) -> list:
        """
        Recupera texto relevante del contenedor de vectores.
        """
        return self.retriever.retrieve(query)
    
    @instrument(
        span_type=SpanAttributes.SpanType.GENERATION)
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Genera respuesta a partir del contexto.
        """
        prompt = f"""
          Eres un asistente experto que extrae información del contexto proporcionado.
          Responde la pregunta de forma extensa, completa y detallada, basándote en el contexto. No alucines.
          Si no tienes la información, simplemente dilo. Si tienes la información que necesitas, solo dime la respuesta.
          Contexto: {context_str}
          Pregunta:
          {query}
          Respuesta:
        """
        response = ""
        stream = complete("mistral-large2", prompt, stream = True)
        for update in stream:    
          response += update
          print(update, end = '')
        return response
    
    @instrument(
        span_type=SpanAttributes.SpanType.RECORD_ROOT, 
        attributes={
            SpanAttributes.RECORD_ROOT.INPUT: "query",
            SpanAttributes.RECORD_ROOT.OUTPUT: "return",
        })
    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)

rag = RAG()

Probemos el sistema RAG consultándolo con una pregunta.

In [ ]:
response = rag.query("¿Cuáles son los casos de uso del servicio Lambda?")

## Registrar la aplicación

Ahora debemos establecer metadatos que incluyan el nombre y la versión de la aplicación, junto con la sesión de Snowpark para almacenar los experimentos.

In [ ]:
from trulens.apps.app import TruApp
from trulens.connectors.snowflake import SnowflakeConnector

tru_snowflake_connector = SnowflakeConnector(snowpark_session=session)

app_name = "aws_search_service_app"
app_version = "1.0.0"

tru_rag = TruApp(
        rag,
        app_name=app_name,
        app_version=app_version,
        connector=tru_snowflake_connector
    )

## Cargar los datos de prueba a Snowflake

Para cargar los datos de prueba que utilizaremos deberá descargar el archivo adjunto como recurso a esta lección `rag_evaluation_dataset.csv` seguir los siguientes pasos para ingestar la información a Snowflake

In [ ]:
-- ============================================================================
-- DDL para tabla de evaluación RAG en Snowflake
-- Propósito: Almacenar preguntas y respuestas de referencia para evaluar 
--           el rendimiento de un sistema RAG basado en servicios AWS
-- ============================================================================

-- Crear la tabla
CREATE OR REPLACE TABLE curso_ia.seccion_6.rag_evaluation_dataset (
    QUERY VARCHAR(500) NOT NULL,
    GROUND_TRUTH_RESPONSE VARCHAR(2000) NOT NULL
)
COMMENT = 'Tabla para almacenar el dataset de evaluación del sistema RAG con preguntas y respuestas de servicios AWS';

-- ============================================================================
-- Comandos para cargar el CSV
-- ============================================================================

-- Crear un file format para el CSV
CREATE OR REPLACE FILE FORMAT curso_ia.seccion_6.csv_format
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    RECORD_DELIMITER = '\n'
    SKIP_HEADER = 1
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    TRIM_SPACE = TRUE
    ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE
    ESCAPE_UNENCLOSED_FIELD = NONE
    COMMENT = 'Formato para cargar archivos CSV del dataset RAG';

-- Crear un stage interno para subir el archivo
CREATE OR REPLACE STAGE curso_ia.seccion_6.rag_dataset_stage
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
    FILE_FORMAT = CSV_FORMAT
    COMMENT = 'Stage interno para cargar el archivo CSV del dataset RAG';


Lugo, debrán subir el archivo `rag_evaluation_dataset.csv` al stage creado a través de Snowsight.

Posteriormente cargaremos la información a la tabla que creamos.

In [ ]:
-- Cargar los datos del CSV a la tabla
COPY INTO curso_ia.seccion_6.rag_evaluation_dataset (QUERY, GROUND_TRUTH_RESPONSE)
FROM @curso_ia.seccion_6.rag_dataset_stage/rag_evaluation_dataset.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

In [ ]:
-- Consultamos la tabla creada
SELECT * FROM curso_ia.seccion_6.rag_evaluation_dataset LIMIT 10;

## Crear un ejecución (Run)

Preparemos la configuración para ejecutar experimentos y agreguemos la ejecución a TruLens.

In [ ]:
from trulens.core.run import Run
from trulens.core.run import RunConfig

run_name = "experimento_1_run"

run_config = RunConfig(
    run_name=run_name,
    dataset_name="curso_ia.seccion_6.rag_evaluation_dataset",
    description="Preguntas sobre servicios de AWS",
    label="aws_rag_eval",
    source_type="TABLE",
    dataset_spec={
        "input": "QUERY",
        "ground_truth_output":"GROUND_TRUTH_RESPONSE",
    },
)

run: Run = tru_rag.add_run(run_config=run_config)

Luego, iniciamos la ejecución del experimento con el conjunto de datos de pruebas preparado. Al hacerlo, se invocará la aplicación en batch utilizando las entradas del conjunto de datos proporcionado durante la ejecución.

In [ ]:
run.start()

## Cálculo de métricas de evaluación

In [ ]:
run.compute_metrics([
    "answer_relevance",
    "context_relevance",
    "groundedness",
])

## Evaluación RAG

![Imagen](https://quickstarts.snowflake.com/guide/getting_started_with_ai_observability/img/2f00466ca5a8244f.png)

### Relevancia del contexto

La relevancia del contexto determina si el contexto obtenido del recuperador o del servicio de búsqueda es relevante para la consulta del usuario. Dada la consulta del usuario y el contexto obtenido, se utiliza un evaluador LLM para determinar la relevancia del contexto obtenido con base en la consulta.

### Fundamentación

La fundamentación determina si la respuesta generada se sustenta y se fundamenta en el contexto obtenido del recuperador o del servicio de búsqueda. Dada la respuesta generada y el contexto obtenido, se utiliza un evaluador LLM para determinar la fundamentación. La implementación subyacente utiliza el razonamiento en cadena de pensamiento para generar las puntuaciones de fundamentación.

### Relevancia de la respuesta

La relevancia de la respuesta determina si la respuesta generada es relevante para la consulta del usuario. Dada la consulta del usuario y la respuesta generada, se utiliza un evaluador LLM para determinar la relevancia de la respuesta al responder a la consulta del usuario. Tenga en cuenta que esto no se basa en la referencia de la respuesta de verdad fundamental y, por lo tanto, no equivale a evaluar la corrección de la respuesta.

## Examinar los resultados

Para ver los resultados de la evaluación, vaya a `Snowsight → IA y ML → Evaluaciones` en el menú lateral.